In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt
import IPython.display as ipd
import wandb

from deepspeech import model, train, datasets, utils, viz, predict

## Train Deepspeech

In [ ]:
dry = False
if dry:
    os.environ['WANDB_MODE'] = 'dryrun'

In [ ]:
p = model.HParams(graphemes=datasets.YESNO_GRAPHEMES)
tp = train.HParams(max_epochs=200, batch_size=32, learning_rate=22e-4)
ds = datasets.YesNo(p)
m = model.DeepSpeech(p)
trainset, testset = datasets.splits(ds, p)

# train
t = train.Trainer(m, trainset, testset, tp)

In [ ]:
t.train()

In [ ]:
viz.ctc_batch(m, [testset[i] for i in range(8)], p)

In [ ]:
yhat, loss = predict.predict_batch(m, [testset[i] for i in range(8)], p)
print(predict.decode_argmax(yhat, p))